# Algoritmos de optimización - Reto 3

Nombre: RAÚL MURILLO GALLEGO<br>
Github: [Repositorio de la actividad](https://github.com/RaulMGallego/03MAIR-Algoritmos-de-Optimizacion-2024/blob/main/ALGORITMOS_R3/Algoritmos_R3_Raul_MurilloGallego.ipynb)<br>
Reto: Mejorar la implementación de recocido simulado implementado en clase sobre el TSP eligiendo una generación de solución vecina con un grado de aleatoriedad menor (función genera_vecino_aleatorio()).

Cambios Principales:

Generación de Vecinos con 2-opt:

- La función genera_vecina_2opt intercambia dos aristas en la ruta, lo que produce soluciones vecinas más estructuradas y potencialmente mejores.

Enfriamiento de la Temperatura:

- La temperatura disminuye multiplicando por un factor de 0.99 en cada iteración, lo que permite una exploración más controlada.

Criterio de Aceptación:

- Se mantiene el criterio de aceptación basado en la probabilidad exponencial, pero ahora se aplica solo cuando la solución vecina es peor que la actual.

**Resultados Esperados:**

La solución final debería ser de mejor calidad que la obtenida con el método anterior, ya que la generación de vecinos con 2-opt tiende a producir soluciones más cercanas al óptimo.

El algoritmo convergerá más rápidamente debido a la reducción de la aleatoriedad en la generación de vecinos.

**Ejemplos de respuestas:**

La mejor solución encontrada es: [18, 8, 9, 23, 40, 24, 21, 39, 22, 38, 29, 30, 32, 28, 27, 2, 3, 6, 1, 0, 34, 33, 20, 35, 36, 31, 17, 7, 37, 15, 16, 14, 19, 5, 26, 13, 4, 10, 41, 25, 12, 11]
Con una distancia total de: **1544**

La mejor solución encontrada es: [21, 40, 24, 39, 22, 38, 29, 30, 27, 28, 32, 35, 36, 17, 31, 20, 33, 34, 0, 1, 7, 15, 37, 16, 14, 19, 13, 5, 6, 26, 18, 4, 3, 2, 8, 12, 11, 10, 25, 41, 23, 9]
Con una distancia total de: **1572**


In [36]:
import urllib.request  # Hacer llamadas HTTP a páginas de la red
import tsplib95       # Módulo para las instancias del problema del TSP
import math           # Módulo de funciones matemáticas
import random         # Para generar valores aleatorios

# Descargamos el fichero de datos (Matriz de distancias)
file = "swiss42.tsp"
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz  # Descomprimir el fichero de datos

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [55]:


# Carga de datos y generación de objeto problem
problem = tsplib95.load(file)

# Nodos
Nodos = list(problem.get_nodes())

# Aristas
Aristas = list(problem.get_edges())

# Función para crear una solución aleatoria
def crear_solucion(Nodos):
    solucion = Nodos.copy()
    random.shuffle(solucion)
    return solucion

# Función para calcular la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.get_weight(a, b)

# Función para calcular la distancia total de una trayectoria/solución
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancia(solucion[i], solucion[i + 1], problem)
    return distancia_total + distancia(solucion[-1], solucion[0], problem)

# Generación de una solución vecina usando 2-opt
def genera_vecina_2opt(solucion):
    # Seleccionar dos índices aleatorios
    i, j = sorted(random.sample(range(1, len(solucion) - 1), 2))
    # Intercambiar las aristas entre i y j
    nueva_solucion = solucion[:i] + solucion[i:j + 1][::-1] + solucion[j + 1:]
    return nueva_solucion

# Función de probabilidad para aceptar peores soluciones
def probabilidad(T, d):
    return random.random() < math.exp(-d / T)

# Función de descenso de temperatura
def bajar_temperatura(T):
    return T * 0.99

# Algoritmo de Recocido Simulado mejorado
def recocido_simulado_mejorado(problem, TEMPERATURA_INICIAL):
    # Solución inicial aleatoria
    solucion_actual = crear_solucion(Nodos)
    distancia_actual = distancia_total(solucion_actual, problem)

    # Mejor solución encontrada
    mejor_solucion = solucion_actual.copy()
    mejor_distancia = distancia_actual

    # Parámetros del recocido simulado
    TEMPERATURA = TEMPERATURA_INICIAL
    while TEMPERATURA > 0.0001:
        # Generar una solución vecina usando 2-opt
        vecina = genera_vecina_2opt(solucion_actual)
        distancia_vecina = distancia_total(vecina, problem)

        # Si la solución vecina es mejor, actualizar la solución actual
        if distancia_vecina < distancia_actual:
            solucion_actual = vecina
            distancia_actual = distancia_vecina
            # Actualizar la mejor solución encontrada
            if distancia_vecina < mejor_distancia:
                mejor_solucion = vecina
                mejor_distancia = distancia_vecina
        else:
            # Aceptar la solución vecina con una probabilidad
            if probabilidad(TEMPERATURA, distancia_vecina - distancia_actual):
                solucion_actual = vecina
                distancia_actual = distancia_vecina

        # Enfriar la temperatura
        TEMPERATURA = bajar_temperatura(TEMPERATURA)

    print("La mejor solución encontrada es:", mejor_solucion)
    print("Con una distancia total de:", mejor_distancia)
    return mejor_solucion

# Ejecutar el recocido simulado mejorado
solucion_final = recocido_simulado_mejorado(problem, TEMPERATURA_INICIAL=1000000)

La mejor solución encontrada es: [21, 40, 24, 39, 22, 38, 29, 30, 27, 28, 32, 35, 36, 17, 31, 20, 33, 34, 0, 1, 7, 15, 37, 16, 14, 19, 13, 5, 6, 26, 18, 4, 3, 2, 8, 12, 11, 10, 25, 41, 23, 9]
Con una distancia total de: 1572
